In [40]:
from string import punctuation
from nltk.corpus import stopwords
from collections import Counter
from scipy import sparse

In [29]:
with open('data/BrasCubas-Assis.txt') as f:
    corpus = f.readlines()
    corpus = [paragraph.rstrip() for paragraph in corpus if paragraph != '\n']

corpus

['Que Stendhal confessasse haver escrito um de seus livros para cem leitores, coisa é que admira e consterna. O que não admira, nem provavelmente consternará, é se este outro livro não tiver os cem leitores de Stendhal, nem cinqüenta, nem vinte, e quando muito, dez. Dez? Talvez cinco. Trata-se, na verdade, de uma obra difusa, na qual eu, Brás Cubas, se adotei a forma livre de um Sterne, ou de um Xavier de Maistre, não sei se lhe meti algumas rabugens de pessimismo. Pode ser. Obra de finado. Escrevia-a com a pena da galhofa e a tinta da melancolia, e não é difícil antever o que poderá sair desse conúbio. Acresce que a gente grave achará no livro umas aparências de puro romance, ao passo que a gente frívola não achará nele o seu romance usual; ei-lo aí fica privado da estima dos graves e do amor dos frívolos, que são as duas colunas máximas da opinião.',
 'Mas eu ainda espero angariar as simpatias da opinião, e o primeiro remédio é fugir a um prólogo explícito e longo. O melhor prólogo é

In [30]:
punctranslation = str.maketrans(dict.fromkeys(punctuation))
set_stopwords = set(stopwords.words('portuguese'))

def tokenize(corpus: str) -> list:
    corpus_tokenized = []
    for paragraph in corpus:
        paragraph = paragraph.lower()
        paragraph = paragraph.encode('utf8', 'ignore').decode()
        paragraph = paragraph.translate(punctranslation)
        corpus_tokenized.append([token for token in paragraph.split() if token not in set_stopwords])
    return corpus_tokenized
    
corpus_tokenized = tokenize(corpus)
corpus_tokenized

[['stendhal',
  'confessasse',
  'haver',
  'escrito',
  'livros',
  'cem',
  'leitores',
  'coisa',
  'admira',
  'consterna',
  'admira',
  'provavelmente',
  'consternará',
  'outro',
  'livro',
  'cem',
  'leitores',
  'stendhal',
  'cinqüenta',
  'vinte',
  'dez',
  'dez',
  'talvez',
  'cinco',
  'tratase',
  'verdade',
  'obra',
  'difusa',
  'brás',
  'cubas',
  'adotei',
  'forma',
  'livre',
  'sterne',
  'xavier',
  'maistre',
  'sei',
  'meti',
  'algumas',
  'rabugens',
  'pessimismo',
  'pode',
  'ser',
  'obra',
  'finado',
  'escreviaa',
  'pena',
  'galhofa',
  'tinta',
  'melancolia',
  'difícil',
  'antever',
  'poderá',
  'sair',
  'desse',
  'conúbio',
  'acresce',
  'gente',
  'grave',
  'achará',
  'livro',
  'umas',
  'aparências',
  'puro',
  'romance',
  'passo',
  'gente',
  'frívola',
  'achará',
  'nele',
  'romance',
  'usual',
  'eilo',
  'aí',
  'fica',
  'privado',
  'estima',
  'graves',
  'amor',
  'frívolos',
  'duas',
  'colunas',
  'máximas',
  'op

In [35]:
unigrams = Counter()
for paragraph in corpus_tokenized:
    for token in paragraph:
        unigrams[token] += 1

token2index = {token: index for index, token in enumerate(unigrams.keys())}
index2token = {index: token for token, index in token2index.items()}

In [38]:
gap = 2
skipgrams = Counter()
for paragraph in corpus_tokenized:
    tokens = [token2index[tok] for tok in paragraph]
    
    for index_word, word in enumerate(paragraph):
        index_context_min = max(0, index_word - gap)
        index_context_max = min(len(paragraph)-1, index_word + gap)

        index_contexts = [index for index in range(index_context_min, index_context_max+1) if index != index_word]
        for index_context in index_contexts:
            skipgram = (tokens[index_word], tokens[index_context])
            skipgrams[skipgram] += 1

most_common = [(index2token[sg[0][0]], index2token[sg[0][1]], sg[1]) for sg in skipgrams.most_common(10)]
most_common

[('alguma', 'coisa', 9),
 ('coisa', 'alguma', 9),
 ('idéia', 'fixa', 7),
 ('fixa', 'idéia', 7),
 ('diminuir', 'diminuir', 6),
 ('mesma', 'coisa', 6),
 ('coisa', 'mesma', 6),
 ('ah', 'brejeiro', 6),
 ('brejeiro', 'ah', 6),
 ('trás', 'trás', 6)]

In [41]:
rows = []
columns = []
data = []

for (token1, token2), sg_count in skipgrams.items():
    rows.append(token1)
    columns.append(token2)
    data.append(sg_count)

ww_matrix = sparse.csr_matrix((data, (rows, columns)))
ww_matrix

<3285x3285 sparse matrix of type '<class 'numpy.int64'>'
	with 22221 stored elements in Compressed Sparse Row format>